# Quantization Demo

## Introduction
In this demo, we will employ PEFT (LoRA) and Quantization techniques to fine-tune the Llama2-7b model, aiming to debias and detoxify text. We will utilize a specific dataset located at `../../data/debiased_profanity_check_with_keywords.csv`.

This notebook will guide you through the process, showcasing the steps involved in fine-tuning the model to produce a debiased and detoxified output from biased or toxic text.

## Importing Libraries
This cell imports libraries for dataset loading.



In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset
import os

## Configuring Directory Paths for Model Weights, Dataset, and Model Storage
This cell specifies the directory paths for storing model checkpoints, adapter models, merged models, and the dataset necessary for the task. We will load the test set to perform inference against the finetuned model.

In [3]:
bucket_name = "jkwng-llama-experiments"
model_dir = "quantized_llama2-7b-chat-hf_20240702164528"
model_bucket_prefix = "llama2"
model_path = f"gs://{bucket_name}/{model_bucket_prefix}/{model_dir}"

In [4]:
#DATASET_PATH = "../../data/debiased_profainty_check_with_keywords.csv" # dataset of biased and corresponding debiased text
DATASET_PATH = f"{model_path}/dataset/test/"


## Creating a HuggingFace Dataset

In [5]:
def create_hf_dataset_from_csv(csv_path):
  dataset = load_dataset('arrow', data_files=csv_path)
  return dataset

dataset = create_hf_dataset_from_csv(f"{DATASET_PATH}data-00000-of-00001.arrow")
#dataset = dataset.train_test_split(test_size=0.1)
dataset = dataset.select_columns(["biased_text", "debiased_text"])


Here are the first 3 samples of the dataset:

In [6]:
print(len(dataset["train"]))

for i in range(3):
    sample = dataset["train"][i]
    print(sample, '\n')

750
{'biased_text': "`` : I do n't think anyone is saying that he is a jackass . It 's about reactions to him , not him , himself . And the next section goes on to list a group of feminist theorists who use psychoanalytic theory.— ``", 'debiased_text': "I don't believe anyone is claiming that he is unpleasant. It is more about the reactions towards him, rather than his own character. Additionally, the next section discusses a group of feminist theorists who utilize psychoanalytic theory."} 

{'biased_text': "its UNAFFORDABLE , its unsustainable , ITS all that and far worst . ...................................... so ? if govt do n't crash economy canada and its society is going towards slow and steady death . O AND DO N'T WORRY ABOUT DEBT SLAVES WITH BIGGER HOUSES AND BIGGER CARS . THEY ARE THE BIGGEST PART OF PROBLEM .", 'debiased_text': 'It is unreasonably expensive, and the current situation is not sustainable. Furthermore, if the government does not take action, Canada and its soci

## Loading Tokenizer

In [7]:
def formatting_prompts_func(examples):
    instruction = (
        " You are a text debiasing bot, you take as input a"
        " text and you output its debiased version by rephrasing it to be"
        " free from any age, gender, political, social or socio-economic"
        " biases, without any extra outputs. Debias this text by rephrasing"
        " it to be free of bias: "
    )
    output_text = []
    for i in range(len(examples["biased_text"])):
        input_text = examples["biased_text"][i]
        response = examples["debiased_text"][i]

        text = f'''Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

        ### Instruction:
        {instruction}

        ### Input:
        {input_text}

        ### Response:
        {response}
        '''

        output_text.append(text)

    return output_text

## Load and Test Trained Model

### Trained Model Generation
Here we test the performance of the trained model:

In [8]:
from google.cloud import aiplatform

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
REGION = os.environ["GOOGLE_CLOUD_REGION"]

In [9]:
endpoint_name = "1398031233740439552"
aip_endpoint_name = (
     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
)
endpoint = aiplatform.Endpoint(aip_endpoint_name)

In [43]:
max_tokens = 256
temperature = 1.0
top_p = 0.9
top_k = 1

In [62]:
instruction = (
    " You are a text debiasing bot, you take as input a"
    " text and you output its debiased version by rephrasing it to be"
    " free from any age, gender, political, social or socio-economic"
    " biases, without any extra outputs. Debias this text by rephrasing"
    " it to be free of bias: "
)

instances = []
predictions = []
references = []
for i in range(100):
  input_text = dataset["train"][i]["biased_text"]
  expected = dataset["train"][i]["debiased_text"]
  references.append(expected)
  #print(input_text)
  text = f'''
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:
  '''

  instances = [
    {
        "prompt": text,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        #"n": 1,
        "stop": ["        "],
        #"stop_token_ids": [-1],
    },
  ]
  response = endpoint.predict(instances=instances)

  for prediction in response.predictions:
    #print(prediction)
    output_pred = prediction.split("Output:\n", 1)[1]
    predictions.append(output_pred)
    #print(f"output: {output_pred}")
    break

  #print(f"expected: {expected}")


In [55]:
!pip install evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 727.5 kB/s eta 0:00:00


In [63]:
# prompt: calculate rouge score

import evaluate

rouge = evaluate.load("rouge", trust_remote_code=True)
results = rouge.compute(predictions=predictions,
                        references=references)
print(results)

{'rouge1': 0.5253818154145768, 'rouge2': 0.32276671219613373, 'rougeL': 0.462837425614692, 'rougeLsum': 0.46812229973405917}
